<a href="https://colab.research.google.com/github/1985shree/1985shree/blob/main/app_defect_validation_using_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Installing and importing modules

In [2]:
# ! pip install spacy

In [4]:
#  ! python -m spacy download en_core_web_lg # download pretrained model

In [6]:
#  import nltk
#  nltk.download('stopwords')

In [7]:
# import regular modules
import pandas as pd
import numpy as np
import json
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
import re
import spacy
nlp = spacy.load('en_core_web_lg')
spacy_tokenizer = Tokenizer(nlp.vocab)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer

nlp = spacy.load('en_core_web_lg')
spacy_tokenizer = Tokenizer(nlp.vocab)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer # to work in parallel on columns
from sklearn.base import BaseEstimator, TransformerMixin # to make fit_transform methods for free 
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# ! unzip -p Data.zip -d data

In [8]:
file_list = os.listdir()
file_names = [n for n in file_list if n[-5:] == '.json']
key_names = [n.replace('.json', "") for n in file_names]
# parsing json files:
data_list = []
for n in file_names:
    with open(n) as fi:
        d = json.load(fi)
        
        data_list.append(d)
        fi.close()

#data_list = [json.load(os.path.join(data_path, n)) for n in file_names]
data_dict = dict(zip(key_names, data_list))


In [9]:
df_cancelled = pd.DataFrame(data_dict['cancelled'])# packing into dataframes
df_noncancelled = pd.DataFrame(data_dict['noncancelled'])
df_sailorapp = pd.DataFrame(data_dict['sailorapp'])

In [10]:
df_cancelled.columns = [c.lower() for c in df_cancelled.columns]
df_noncancelled.columns = [c.lower() for c in df_noncancelled.columns]
df_sailorapp.columns = [c.lower() for c in df_sailorapp.columns]

In [11]:
# extracting details of the comments from json fomat data in commnets column
df_cancelled['cmt_details'] = df_cancelled.comments.apply(lambda x: [i['body'] for i in x])
df_cancelled['cmt_details'] = df_cancelled.cmt_details.apply(lambda x: "".join(x))

df_noncancelled['cmt_details'] = df_noncancelled.comments.apply(lambda x: [i['body'] for i in x])
df_noncancelled['cmt_details'] = df_noncancelled.cmt_details.apply(lambda x: "".join(x))

df_sailorapp['cmt_details'] = df_sailorapp.comments.apply(lambda x: [i['body'] for i in x])
df_sailorapp['cmt_details'] = df_sailorapp.cmt_details.apply(lambda x: "".join(x))

In [12]:
df_cancelled.cmt_details[1]

'Cancelling the defect because there was slight delay(expected) between ship to shore sync.  Hence the count difference.  Now the data has caught up and the counts are matching.'

In [13]:
df_cancelled['status'] = 0
df_noncancelled['status'] = 1 # setting the status target for each

In [14]:
# preparing df for train test split
df = pd.concat([df_cancelled, df_noncancelled]) # concatenate 2 dataframes into one

In [15]:
df.shape

(5689, 13)

In [16]:
df.head()

,status,description,comments,priority,rootcause,reporter,issuekey,summary,sprint,component,application,assigne,cmt_details
0,0,"Build: PI7.6 FE + 8.2 BE Hotfix , App Version:...",[],P3 - Medium,NA,Anusha Boosa,MSH-45790,CERT|SailorAPP| On-board | Cabin Services | I...,,NA,Sailor App,Anusha Boosa,
1,0,Ship parse count for housekeeping requests:\r\...,[{'author': {'accountId': '5e8b26768764110b816...,P3 - Medium,Code,Shanthi K,MSH-45793,Counts in the parse tables are not matching be...,PI8.4,NA,Crew VXP,Anagha Sebastian,Cancelling the defect because there was slight...
2,0,Hi [~accountid:5ade5093027e7a2ebfcbd97e] / [~a...,[{'author': {'accountId': '5ade5093027e7a2ebfc...,P2 - High,NA,Kapil Soni,MSH-45795,Magnolia | KAFKA | Ship Side Events are not co...,,NA,Sailor App,Kapil Soni,[~accountid:5d4c220bc54e920cf3b81002] - can yo...
3,0,Ship side team has successfully uploaded the s...,[{'author': {'accountId': '5caee6ee6c3b6b270f4...,P2 - High,NA,Exalate,MSH-45830,VV-Prod-Ship | VM - Bulk uploaded successfully...,,"""Crew VXP""",Crew VXP,Himanshu Kumawat,FYI: [DC Support|https://virginvoyages.atlassi...
4,0,Hit the application url and do the booking flo...,[{'author': {'accountId': '5dcdc9960e14000dd49...,P4 - Low,NA,Punithavalli,MSH-45840,DWP-QA-Spinner getting displayed in the top le...,,NA,eCommerce,Punithavalli,[~accountid:5d9b8c9033b8de0dc69a4f9a] I don’t ...


In [17]:
countvectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
svc_classifier = LinearSVC()
vanilla_logistic = LogisticRegression()
tfidfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer) # we will use both vectorizers

In [18]:
def clean_string(text,stem="None"):

    final_string = ""

    # make lower
    text = text.lower()
    
    # remove http
    text = re.sub(r"http.*\.[a-z]{2,3}","",text)

    # remove www
    text = re.sub(r"www.*\.[a-z]{2,3}","",text)
    
    # remove line breaks
    text = re.sub(r'\n','',text)

    # remove puncuation
    translator = str.maketrans('','',string.punctuation)
    text = text.translate(translator)

    # remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words.remove("don't")
    useless_words.remove('not')

    text_filtered = [word for word in text if not word in useless_words]

    # remove numbers
    text_filtered = [re.sub(r'\w*\d\w*','',w) for w in text_filtered]

    # stem or lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string
        
def apply_clean_string(df):
   arr = df.apply(lambda x: clean_string(str(x)))
   return pd.DataFrame(arr)  

forget pipelines let me try this first

In [ ]:
# df['description'] = df.description.apply(lambda x: clean_string(x))
# # df['comments'] = df.comments.apply(lambda x: clean_string(x))
# df['summary'] = df.summary.apply(lambda x: clean_string(x))
# # df['component'] = df.component.apply(lambda x: clean_string(x))
# df['cmt_details'] = df.cmt_details.apply(lambda x: clean_string(x))

In [19]:
df['soup'] = df['description'] + df['summary'] +df['application']+ df['component'] + df['cmt_details']

In [51]:
# checking individual features

In [39]:
df['sumary'] = df.summary.apply(lambda x: clean_string(x)) # this gives a better score, models not so important, try features, categories too

In [40]:
count_pipe = Pipeline([
                 ('vectorizer', countvectorizer),
                 ('classifier', svc_classifier)])

In [ ]:
# count_pipe_logistic = Pipeline([
#                  ('vectorizer', countvectorizer),
#                  ('classifier', vanilla_logistic)])

In [41]:
tf_pipe = Pipeline([
                 ('vectorizer', tfidfvectorizer),
                 ('classifier', svc_classifier)])

In [42]:
X = df['summary']
y = df[['status']]

In [43]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=111)# test 

In [44]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=111)# train-val

In [45]:
count_pipe.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('vectorizer',
                 CountVectorizer(tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f1b21b70ca0>)),
                ('classifier', LinearSVC())])

In [46]:
count_pipe_val = count_pipe.predict(X_val)
count_val_score = accuracy_score(count_pipe_val, y_val)


In [47]:
count_val_score

0.575192096597146

In [32]:
# count_pipe_logistic.fit(X_train, y_train)

In [33]:
# count_pipe_logistic_val = count_pipe_logistic.predict(X_val)
# count_logistic_val_score = accuracy_score(count_pipe_logistic_val, y_val)


In [34]:
# count_logistic_val_score

In [48]:
tf_pipe.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f1b21b70ca0>)),
                ('classifier', LinearSVC())])

In [49]:
tf_pipe_val = tf_pipe.predict(X_val)
tf_val_score = accuracy_score(tf_pipe_val, y_val)

In [50]:
tf_val_score

0.575192096597146

making a pipeline by plugging in the text cleaning step

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

naive_bayes = MultinomialNB()
knn = KNeighborsClassifier()


nb_pipe = Pipeline([('vectorizer', tfidfvectorizer),
                 ('classifier', svc_classifier)])
params = {'classifier__C': [1, 10, 100]}
        
         
tf_grid = GridSearchCV(tf_pipe, param_grid = params, cv=4,
                               n_jobs=-3, verbose=1)

nb_grid = GridSearchCV(nb_pipe, param_grid = params, cv=4,
                               n_jobs=-3, verbose=1)

In [65]:
nb_grid.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__vectorizer', 'estimator__classifier', 'estimator__vectorizer__analyzer', 'estimator__vectorizer__binary', 'estimator__vectorizer__decode_error', 'estimator__vectorizer__dtype', 'estimator__vectorizer__encoding', 'estimator__vectorizer__input', 'estimator__vectorizer__lowercase', 'estimator__vectorizer__max_df', 'estimator__vectorizer__max_features', 'estimator__vectorizer__min_df', 'estimator__vectorizer__ngram_range', 'estimator__vectorizer__norm', 'estimator__vectorizer__preprocessor', 'estimator__vectorizer__smooth_idf', 'estimator__vectorizer__stop_words', 'estimator__vectorizer__strip_accents', 'estimator__vectorizer__sublinear_tf', 'estimator__vectorizer__token_pattern', 'estimator__vectorizer__tokenizer', 'estimator__vectorizer__use_idf', 'estimator__vectorizer__vocabulary', 'estimator__classifier__C', 'estimator__classifier__class_weight', 'estimator__classifier__dual', 'e

In [52]:
nb_grid.fit(X_train_val, y_train_val)

Fitting 4 folds for each of 3 candidates, totalling 12 fits


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f1b21b70ca0>)),
                                       ('classifier', LinearSVC())]),
             n_jobs=-3, param_grid={'classifier__C': [1, 10, 100]}, verbose=1)

In [53]:
nb_grid.best_estimator_

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f1aea6a7a60>)),
                ('classifier', LinearSVC(C=1))])

In [54]:
nb_grid.best_score_

0.5706438489349304

In [ ]:
df.priority.value_counts() # use cat encoder..one hot?

P2 - High      2431
P3 - Medium    1841
P1 - Urgent    1157
P4 - Low        260
Name: priority, dtype: int64

In [ ]:
df.rootcause.value_counts() # use cat encoder...label?

NA                     3952
Code                    950
Environmental           164
Data                    155
Content                 135
Design                  105
Invalid Issue           101
Requirements             60
Database                 45
User Error               11
Product Limitations       6
Hardware                  5
Name: rootcause, dtype: int64

In [ ]:
X = df[['description', 'rootcause',
         'component', 'summary',
        'cmt_details']]
y = df[['status']]

In [ ]:
X['decsription']= X.description.apply(lambda x: countvectorizer.fit_transform(x))
X['rootcause'] = X.rootcause.apply(lambda x: countvectorizer.fit_transform(x))
X['component'] = X.component.apply(lambda x: countvectorizer.fit_transform(x))
X['summary'] = X.summary.apply(lambda x: countvectorizer.fit_transform(x))
X['cmt_details'] = X.cmt_details.apply(lambda x: countvectorizer.fit_transform(x))

ValueError: ignored

description      (0, 0)\t1\n  (0, 7610)\t1\n  (0, 14576)\t1\n...
rootcause        (0, 0)\t1\n  (1, 1)\t1\n  (2, 2)\t1\n  (3, 3...
summary          (0, 0)\t1\n  (0, 16602)\t1\n  (0, 24791)\t1\...
component        (0, 0)\t1\n  (1, 1)\t1\n  (2, 2)\t1\n  (3, 3...
cmt_details      (1, 0)\t1\n  (1, 8887)\t1\n  (1, 10513)\t1\n...
dtype: object

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=111)# test 

ValueError: ignored

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=111)# train-val

In [ ]:
X_train.shape

(3640, 5)

In [ ]:
y_train.shape

(3640, 1)

In [ ]:
# class MyLabelEncoder(TransformerMixin):
#     def __init__(self, *args, **kwargs):
#         self.encoder = LabelEncoder()
#     def fit(self, x, y=0):
#         self.encoder.fit(x)
#         return self
#     def transform(self, x, y=0):
#         return self.encoder.transform(x)

In [ ]:
ohe = OneHotEncoder()
lab = LabelEncoder()
# textencoder = FunctionTransformer(lambda array: apply_custom_standardization(array))

In [ ]:
# preprocessor = ColumnTransformer([
#     ('one_hot', ohe, ['priority'])],
#     remainder='passthrough'
)

ColumnTransformer(remainder='passthrough',
                  transformers=[('one_hot', OneHotEncoder(), ['priority'])])

In [ ]:
# countvec = ColumnTransformer([
#     ('vectorizer', countvectorizer, ['description', 'rootcause',
#          'summary', 'component', 
#         'cmt_details'])],
#     remainder='passthrough'
# )

In [ ]:
# # pipe = Pipeline([("preprocessor", preprocessor),
#                  ('vectorizer', countvec),
#                  ('classifier', svc_classifier)])

In [ ]:
# pipe

Pipeline(steps=[('vectorizer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vectorizer',
                                                  CountVectorizer(tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f7d8a281040>),
                                                  ['description', 'rootcause',
                                                   'summary', 'component',
                                                   'cmt_details'])])),
                ('classifier', LinearSVC())])

In [ ]:
X_train.columns

Index(['description', 'rootcause', 'summary', 'component', 'cmt_details'], dtype='object')

In [ ]:
print(len(X_train), len(y_train))

3640 3640


In [ ]:
pipe.fit(X_train,y_train) 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

ValueError: ignored

ColumnTransformer(transformers=[('one_hot_encoder', OneHotEncoder(),
                                 ['priority']),
                                ('labelencoder', LabelEncoder(),
                                 ['rootcause', 'component'])])